# Description

## Import libaraies

In [1]:
import keras.optimizers
import numpy as np, os, pathlib, matplotlib.pyplot as plt, sys, seaborn as sns
from keras.layers import Input, Dense, BatchNormalization, Dropout, Flatten, Reshape, Lambda, Concatenate
from keras.models import Model
from keras.layers import concatenate
from keras.layers import Rescaling, Reshape, Resizing, RandomZoom, RandomRotation, RandomFlip

import tensorflow as tf
from keras.metrics.metrics import binary_crossentropy
from keras.layers import LeakyReLU
from keras import backend as bk
# from tensorflow.keras.utils import to_categorical
import parameters as p

In [2]:
# from tensorflow.python.framework.ops import enable_eager_execution
# enable_eager_execution()

## Parameters

In [8]:
# PARAMETERS
batch_size = 240
latent_dim = 512  # to be easier generate and visualize result
dropout_r = 0.1
lr_0 = 0.0001
epoch = 10

img_height = 224
img_width = 224
ims = 224
ishape = (ims, ims, 3)
ishape_out = (ims, ims, 1)

Adam = keras.optimizers.Adam

In [4]:
name = f'cats_cvae_dim{latent_dim}_epochs{epoch}_ims{ims}'

data_dir = p.dataset_folder
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
print('Number of images:', image_count)

Number of images: 4800


In [6]:
if batch_size < image_count:
    batch_size = int(image_count * 0.05)
    # print(batch_size)
else:
    batch_size = image_count // 2

print('BATCH SIZE:', batch_size)
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

BATCH SIZE: 240


In [9]:
train_img_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    batch_size=batch_size,
    image_size=(ims, ims),
    labels=None,
    color_mode="rgb"
)

valid_img_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    batch_size=batch_size,
    image_size=(ims, ims),
    color_mode="rgb")

Found 4800 files belonging to 2 classes.
Using 3840 files for training.
Found 4800 files belonging to 2 classes.
Using 960 files for validation.


In [ ]:
train_img_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    batch_size=batch_size,
    image_size=(ims, ims),
    labels=None,
    color_mode="rgb"
)

valid_img_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    batch_size=batch_size,
    image_size=(ims, ims),
    color_mode="rgb")

In [10]:
class_names = train_ds.class_names  # os.listdir(os.path.join(data_dir))
num_classes = len(class_names)
print('num_classes:', num_classes, '-', class_names)


full_imgs = lambda ds: np.concatenate([x for x, y in ds])
full_lbls = lambda ds: np.concatenate([y for x, y in ds])

train_img = full_imgs(train_ds)
train_img = train_img.astype('float32') / 255.
train_lbl = full_lbls(train_ds)
# train_lbl_cat = tf.keras.utils.to_categorical(train_lbl).astype(np.float32)

valid_img = full_imgs(valid_ds)
valid_img = valid_img.astype('float32') / 255.
valid_lbl = full_lbls(valid_ds)
print('\n\nshape train_img, train_lbl:', np.shape(train_img), np.shape(train_lbl), '\n')

num_classes: 2 - ['cats', 'dogs']


shape train_img, train_lbl: (3840, 224, 224, 3) (3840, 2) 

